In [1]:
import numpy as np
from galois_datasets import load_all_data_mnist, load_all_data_cifar10, load_all_data_fashion_mnist
from utils import create_batch_data, to_finite_field_domain, from_galois_to_real_domain
import modules
import galois_layers
from galois_criterions import GaloisFieldMSELoss
from sklearn.datasets import make_classification
import galois

/home/umityigitbsrn/miniconda3/envs/pytorch-stable/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
BATCH_SIZE = 256
EPOCH = 100
PRINT = 1
FLATTEN = False
# 0, MNIST; 1, FashionMNIST; 2, CIFAR10; 3 RANDOM
DATASET_MODE = 3
PRIME = 2**26 - 5
QUANTIZATION_WEIGHT = 8
QUANTIZATION_INPUT = 8
QUANTIZATION_BATCH_SIZE = 8
LR = 1

In [3]:
field = galois.GF(PRIME)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [4]:
# data fetching
load_path = '../../data'
if DATASET_MODE == 0:
    train_data, train_label, test_data, test_label = load_all_data_mnist(load_path, QUANTIZATION_INPUT, QUANTIZATION_WEIGHT, PRIME, field, flatten=FLATTEN)
elif DATASET_MODE == 1:
    train_data, train_label, test_data, test_label = load_all_data_fashion_mnist(load_path, QUANTIZATION_INPUT, QUANTIZATION_WEIGHT, PRIME, field, flatten=FLATTEN)
elif DATASET_MODE == 2:
    train_data, train_label, test_data, test_label = load_all_data_cifar10(load_path, QUANTIZATION_INPUT, QUANTIZATION_WEIGHT, PRIME, field, flatten=FLATTEN)
elif DATASET_MODE == 3:
    train_data, train_label = make_classification(n_samples=25, n_features=28*28*3, n_classes=10, n_clusters_per_class=1, n_informative=120)
    train_data = train_data.reshape((-1, 3, 28, 28))
    test_data, test_label = train_data, train_label
    train_label = np.zeros((25, 10))
    for idx, label in enumerate(test_label):
        train_label[idx][label] = 1
    train_data, train_label, test_data = to_finite_field_domain(train_data, QUANTIZATION_INPUT, PRIME), to_finite_field_domain(train_label, QUANTIZATION_WEIGHT, PRIME), to_finite_field_domain(test_data, QUANTIZATION_INPUT, PRIME)
    train_data, train_label, test_data = field(train_data), field(train_label), field(test_data)
else:
    train_data, train_label, test_data, test_label = None, None, None, None
train_data, train_label, test_data, test_label = create_batch_data(train_data, train_label, test_data, test_label, BATCH_SIZE)

In [5]:
model_arr = [
    galois_layers.GaloisFieldPiNetSecondOrderConvLayer(3, 6, (5, 5), QUANTIZATION_WEIGHT, PRIME, field, first_layer=True, padding=(2, 2, 2, 2), quantization_bit_input=QUANTIZATION_INPUT),
    galois_layers.GaloisFieldPiNetSecondOrderConvLayer(6, 16, (5, 5), QUANTIZATION_WEIGHT, PRIME, field, first_layer=True, quantization_bit_input=QUANTIZATION_INPUT),
    modules.Flatten(),
    galois_layers.GaloisFieldPiNetSecondOrderLinearLayer(9216, 128, QUANTIZATION_WEIGHT, PRIME, field),
    galois_layers.GaloisFieldLinearLayer(128, 10, QUANTIZATION_WEIGHT, PRIME, field)
]

model = modules.Network(model_arr)
criterion = GaloisFieldMSELoss(PRIME, QUANTIZATION_WEIGHT, QUANTIZATION_BATCH_SIZE, field)

In [6]:
for epoch in range(EPOCH):
    tot_loss = 0
    for train_idx, (train_data_batch, train_label_batch) in enumerate(zip(train_data, train_label)):
        # train
        preds = model.forward(train_data_batch)

        loss = criterion.forward(preds, train_label_batch)
        tot_loss += loss
        propagated_error = criterion.error_derivative()

        model.backprop(propagated_error)
        model.optimize(LR)
        print(loss)
        # if (train_idx + 1) % PRINT == 0:
        #     tot_acc = 0
        #     tot_sample = 0
        #     for train_acc_idx, (test_data_batch, test_label_batch) in enumerate(zip(test_data, test_label)):
        #         # train accuracy
        #         preds = model.forward(test_data_batch)
        #         preds = from_galois_to_real_domain(preds, QUANTIZATION_WEIGHT, PRIME)
        #         pred_args = np.argmax(preds, axis=1)
        #
        #         tot_acc += np.count_nonzero(pred_args == test_label_batch)
        #         tot_sample += test_data_batch.shape[0]
        #     accuracy = tot_acc / tot_sample
        #     if train_idx != 0:
        #         tot_loss = tot_loss / PRINT
        #     print('epoch: {}, idx: {}, accuracy: {}, loss: {}'.format(epoch + 1, train_idx + 1, accuracy, tot_loss))
        #     tot_loss = 0

15651.882026977539
505690.41200073244


KeyboardInterrupt: 